遍歷實驗資料，抓出電壓校正離群直並印出結果

In [2]:
# import used functioons 
#  import tools 
%matplotlib inline
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Conv1D, Dense, concatenate, RepeatVector, MaxPooling1D, Activation ,UpSampling1D, Conv1DTranspose
from tensorflow.keras.layers import Add, Concatenate,LSTM, TimeDistributed, MultiHeadAttention, LayerNormalization


from tensorflow.keras.utils import plot_model

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPU devices found:")
    for gpu in gpus:
        print(gpu)
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

import numpy as np 
import pandas as pd
from openpyxl import Workbook
import matplotlib.pyplot as plt
import os
import sys
import csv
import time
from IPython import embed

from rul_features.rul_data_read import read_rul_data

from test_algs.CCAE_ntu_rul import get_initial_files_datalist, CCAE_train
from test_algs.CCAE_ntu_rul import CCAE_model_application, CCAE_model_build_train

import random
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(42)

# Initial model parameters
sequence_length=1024
layer_number=1
future_length=512
#變數宣告
Fs=20000
Rs=12.5
P=4

model_folder_path = r'NTU_rul_models\CCAE_models'
model_name = f'0823_multiscale_CCAE_Base{sequence_length}Future{future_length}'
model_path = os.path.join(model_folder_path, model_name)
os.makedirs(model_path, exist_ok=True)
my_model_V_in_I_out= load_model(os.path.join(model_path, model_name + '_VI.keras'))
# my_model_V_in_I_out.summary()

GPU devices found:
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [3]:
# 資料增強
import re,shutil
def data_random_augmentation(raw_data, sequence_length, augmen_sample_number=1000, future_length=64):
    
    # 增加一個回傳時間步長，作為下一點訓練資料使用
    sequence_length=sequence_length+future_length  
    
    # 從 原始資料df 中隨機選取augmen_sample_number 個起始點為sequence_length長度的序列樣本。
    # raw_data 為 n,4 的序列資料，每一行分別為 voltage alpha, voltage beta, current alpha, current beta
    #資料增強倍率
    max_augment_factor = len(raw_data)-sequence_length+1
    # 最多增強樣本數不能超過資料長度除以序列長度
    augmen_sample_number = min(augmen_sample_number, max_augment_factor)


    # 初始化一個空的列表來存放提取出的樣本數據  
    samples_list = []

    # 隨機選取 augmen_sample_number 個起始點
    start_indices = np.random.choice(len(raw_data) - sequence_length + 1, augmen_sample_number, replace=False)
    samples_list = [ raw_data[start_idx:start_idx+sequence_length][:] for start_idx in start_indices]

    # 將收集到的所有樣本轉換成 NumPy 多維陣列
    final_data = np.array(samples_list)
    
    return final_data

# 確認離群直
def list_model_MSEs(Normal_subfolders, model, test_size=0.5):
    # 列出目標資料夾下的檔案重建誤差以找出離群值
    augmen_sample_number=100
    """取得指定工況資料夾下的所有檔案，並找出離群值"""
    parquet_files = [os.path.join(Normal_subfolders, f) for f in os.listdir(Normal_subfolders) if f.endswith(".parquet")]
    if len(parquet_files)>1:
        apply_parquet_files, _ = train_test_split(parquet_files, test_size=test_size)
    else:
        apply_parquet_files = parquet_files


    for file_path in apply_parquet_files:
        train_signals=[] # vi signals alpha beta
        train_signals_normalized=[]
        torque_array=[]
        speed_array_pu=[]
        if not os.path.exists(file_path):
            print(f"File {file_path} does not exist.")
            continue
        # 讀取資料
        df = read_rul_data(file_path, force_recompute=True)
        if df["Voltage alpha thd"] > 0.1 or df["Voltage beta thd"] > 0.1 or df is None:
            print(f"File {file_path} has high THD, skipping.")
            continue
        current_alpha= np.array(df["Current alpha downsample"])
        current_beta= np.array(df["Current beta downsample"])
        flux_alpha= np.array(df["Flux alpha"])
        flux_beta= np.array(df["Flux beta"])
        if (flux_alpha.shape[0] != current_alpha.shape[0]):
            print(f"File {file_path} has mismatched flux and current lengths, skipping.")
            print(f"flux_alpha length: {flux_alpha.shape[0]}, current_alpha length: {current_alpha.shape[0]}")
            continue
        torque_airgap=1.5*4*(flux_alpha*current_beta-flux_beta*current_alpha)  # 計算 torque
        emf_alpha= (df["Voltage alpha downsample"] - Rs * current_alpha) / (df["Speed"][0] * P * 2 * np.pi / 60)  # 計算電動勢
        emf_beta= (df["Voltage beta downsample"] - Rs * current_beta) / (df["Speed"][0] * P * 2 * np.pi / 60)
        # 標準化
        torque_airgap_normalized = (torque_airgap - np.mean(torque_airgap)) / np.std(torque_airgap)
          
        torque= (df["Torque avg"]) 
        # print(df)
        speed= (df["Speed"][0])
        
        # 合併測試資料至訓練維度 shape=(,4000,dim)
        temp_train_signals = [np.array(df["Voltage alpha downsample"]), 
                              np.array(df["Voltage beta downsample"]),
                              current_alpha,
                              current_beta,
                              flux_alpha,
                              flux_beta,
                              emf_alpha,
                              emf_beta,
                              torque_airgap]
        
        # 將數據正規化到 [-1, 1] 範圍
        temp_train_signals_normalized = []
        # for signal in temp_train_signals:
        #     min_val = np.min(signal)
        #     max_val = np.max(signal)
        #     # 避免除以零的狀況（max == min）
        #     if max_val == min_val:
        #         normalized = np.zeros_like(signal)
        #     else:
        #         normalized = 2 * (signal - min_val) / (max_val - min_val) - 1
        #     temp_train_signals_normalized.append(normalized)
            
        # 將數據標準化（均值為 0，標準差為 1）
        
        for signal in temp_train_signals:
            mean = np.mean(signal)
            std = np.std(signal)
            # 避免除以零的狀況（std == 0）
            if std == 0:
                standardized = np.zeros_like(signal)
            else:
                standardized = (signal - mean) / std
            temp_train_signals_normalized.append(standardized)
        
        
        temp_train_signals_normalized[-1]= torque_airgap_normalized  # 改成std正規化的 torque_airgap
        
        try:
            temp_train_signals = np.stack(temp_train_signals, axis=1)
            temp_train_signals_normalized = np.stack(temp_train_signals_normalized, axis=1)
        except ValueError as e:
            print(f"Error stacking signals for file {file_path}: {e}")
            print(f"datalength {len(df['Voltage alpha downsample'])}")
            continue
        
        temp_train_input_signals_augmented = data_random_augmentation(temp_train_signals, 1024, augmen_sample_number=100,future_length=512)
        temp_train_signals_normalized_augmented = data_random_augmentation(temp_train_signals_normalized, 1024, augmen_sample_number=100,future_length=512)
        train_signals.append(temp_train_input_signals_augmented)
        train_signals_normalized.append(temp_train_signals_normalized_augmented)
        torque_array.append(np.ones(augmen_sample_number) * torque)
        speed_array_pu.append(np.ones(augmen_sample_number) * speed/3000) # rate 3000 rpm 
        
        train_signals = np.concatenate(train_signals, axis=0).astype(np.float32)
        train_signals_normalized = np.concatenate(train_signals_normalized, axis=0).astype(np.float32)
        torque_array = np.concatenate(torque_array, axis=0).astype(np.float32)
        speed_array_pu = np.concatenate(speed_array_pu, axis=0).astype(np.float32)
        
        apply_input_data = train_signals_normalized[:, :sequence_length, 0:2]  # 過去 emf
        apply_output_data = train_signals_normalized[:, :sequence_length, 2:4]  # 過去 current（僅用於比較/誤差）
        check_reconstruction_normal = model.predict(
            [apply_input_data,
            speed_array_pu.reshape(-1, 1),
            torque_array.reshape(-1, 1)],
            batch_size=128,
            verbose=0
        )

        error = apply_output_data[:, :, 0] - check_reconstruction_normal[:, :, 0]
        error_mse = np.mean(np.square(error),axis=1)  # 計算每個樣本的 MSE
        print(error_mse.mean(), end='')
        print(file_path)

    return error_mse

def list_voltage_thd(Normal_subfolders):
    # 列出目標資料夾下的檔案重建誤差以找出離群值
    """取得指定工況資料夾下的所有檔案，並找出離群值"""
    # 在 Normal_subfolders 建立子資料夾 voltage_plot
    voltage_plot_folder = os.path.join(Normal_subfolders, "voltage_plot")
    outlier_folder = os.path.join(Normal_subfolders, "outlier")
    if not os.path.exists(voltage_plot_folder):
        os.makedirs(voltage_plot_folder)
    if not os.path.exists(outlier_folder):
        os.makedirs(outlier_folder)
    # 如果 outlier_folder 內已有檔案則直接跳過
    if len(os.listdir(outlier_folder)) > 0:
        print(f"Outlier folder {outlier_folder} is not empty, skipping.")
        return []
    
    # 依檔案編號排序
    parquet_files = [os.path.join(Normal_subfolders, f) for f in os.listdir(Normal_subfolders) if f.endswith(".parquet")]
    parquet_files = sorted(parquet_files, key=lambda x: int(re.search(r"(\d+)\.parquet$", os.path.basename(x)).group(1)))
    print(parquet_files)
    voltage_thd_list = []
    alpha_thd_list=[]
    for file_path in parquet_files:
        if not os.path.exists(file_path):
            print(f"File {file_path} does not exist.")
            continue
        # 讀取資料
        df = read_rul_data(file_path, force_recompute=True)
        if df is None:
            print(f"File {file_path} could not be read, skipping.")
            continue
        voltage_alpha = np.array(df["Voltage alpha downsample"])
        voltage_beta = np.array(df["Voltage beta downsample"])
        voltage_alpha_thd = df["Voltage alpha thd"]
        voltage_beta_thd = df["Voltage beta thd"]
        
        # 畫圖並儲存
        plt.figure(figsize=(12, 6))
        plt.plot(voltage_alpha, label=f'Voltage Alpha thd: {voltage_alpha_thd[0]:.4f}')
        plt.plot(voltage_beta, label=f'Voltage Beta thd: {voltage_beta_thd[0]:.4f}')
        plt.title(f'Voltage Alpha/Beta: {os.path.basename(file_path)}')
        plt.xlabel('Sample')
        plt.ylabel('Voltage')
        plt.legend()
        save_path = os.path.join(voltage_plot_folder, os.path.splitext(os.path.basename(file_path))[0] + '_voltage.png')
        plt.savefig(save_path)
        plt.close()
        alpha_thd_list.append(voltage_alpha_thd[0])
        voltage_thd_list.append((os.path.splitext(os.path.basename(file_path))[0], voltage_alpha_thd[0], voltage_beta_thd[0]))
        # print(f"File: {file_path}, Voltage Alpha THD: {voltage_alpha_thd}, Voltage Beta THD: {voltage_beta_thd}")
    
    # 找出 alpha_thd_list 最大的五個值的索引並將對應的翻轉不好的電壓檔案移動到 outlier_folder
    if len(alpha_thd_list) >= 5:
        alpha_thd_array = np.array(alpha_thd_list)
        top5_indices = alpha_thd_array.argsort()[-5:][::-1]
        for idx in top5_indices:
            src_file = parquet_files[idx]
            dst_file = os.path.join(outlier_folder, os.path.basename(src_file))
            print(f"Moving outlier file: {src_file} -> {dst_file}")
            shutil.move(src_file, dst_file)
            
    # 將 voltage_thd_list 存到 CSV
    csv_save_path = os.path.join(voltage_plot_folder, "voltage_thd_list.csv")
    with open(csv_save_path, mode='w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['file_path', 'voltage_alpha_thd', 'voltage_beta_thd'])
        for row in voltage_thd_list:
            writer.writerow(row)
    return voltage_thd_list

test_folder=r"D:\OneDrive\CCAE_experiments\CCAE_datasets3_1\Normal1-3"
test_subfolders = [f.path for f in os.scandir(test_folder) if f.is_dir()]
for subfolders in test_subfolders:
    _=list_voltage_thd(subfolders)


Outlier folder D:\OneDrive\CCAE_experiments\CCAE_datasets3_1\Normal1-3\Normal1 1200 0.5v 2 kg cm2\outlier is not empty, skipping.
Outlier folder D:\OneDrive\CCAE_experiments\CCAE_datasets3_1\Normal1-3\Normal1 1200 1.0 v 2 kg cm2\outlier is not empty, skipping.
Outlier folder D:\OneDrive\CCAE_experiments\CCAE_datasets3_1\Normal1-3\Normal1 1200 2.0v 2 kg cm2\outlier is not empty, skipping.
Outlier folder D:\OneDrive\CCAE_experiments\CCAE_datasets3_1\Normal1-3\Normal1 1800 0.5v 2 kg cm2\outlier is not empty, skipping.
Outlier folder D:\OneDrive\CCAE_experiments\CCAE_datasets3_1\Normal1-3\Normal1 1800 1.0v 2 kg cm2\outlier is not empty, skipping.
Outlier folder D:\OneDrive\CCAE_experiments\CCAE_datasets3_1\Normal1-3\Normal1 1800 2.0v 2 kg cm2\outlier is not empty, skipping.
Outlier folder D:\OneDrive\CCAE_experiments\CCAE_datasets3_1\Normal1-3\Normal1 3000 0.5v 2 kg cm2\outlier is not empty, skipping.
Outlier folder D:\OneDrive\CCAE_experiments\CCAE_datasets3_1\Normal1-3\Normal1 3000 1.0v 

強制計算RUL數據

In [6]:
list_voltage_thd(r"D:\OneDrive\RUL HI Reasearch Result\Data_sets\NTU_RUL_v2_data\Acc_life_test_data\Uncategorized_Data\0722_1800_10p_1.0v_1_1")

['D:\\OneDrive\\RUL HI Reasearch Result\\Data_sets\\NTU_RUL_v2_data\\Acc_life_test_data\\Uncategorized_Data\\0722_1800_10p_1.0v_1_1\\RUL_Data_3_24.parquet', 'D:\\OneDrive\\RUL HI Reasearch Result\\Data_sets\\NTU_RUL_v2_data\\Acc_life_test_data\\Uncategorized_Data\\0722_1800_10p_1.0v_1_1\\RUL_Data_3_25.parquet', 'D:\\OneDrive\\RUL HI Reasearch Result\\Data_sets\\NTU_RUL_v2_data\\Acc_life_test_data\\Uncategorized_Data\\0722_1800_10p_1.0v_1_1\\RUL_Data_3_26.parquet', 'D:\\OneDrive\\RUL HI Reasearch Result\\Data_sets\\NTU_RUL_v2_data\\Acc_life_test_data\\Uncategorized_Data\\0722_1800_10p_1.0v_1_1\\RUL_Data_3_27.parquet', 'D:\\OneDrive\\RUL HI Reasearch Result\\Data_sets\\NTU_RUL_v2_data\\Acc_life_test_data\\Uncategorized_Data\\0722_1800_10p_1.0v_1_1\\RUL_Data_3_28.parquet', 'D:\\OneDrive\\RUL HI Reasearch Result\\Data_sets\\NTU_RUL_v2_data\\Acc_life_test_data\\Uncategorized_Data\\0722_1800_10p_1.0v_1_1\\RUL_Data_3_29.parquet', 'D:\\OneDrive\\RUL HI Reasearch Result\\Data_sets\\NTU_RUL_v2_d

[('RUL_Data_3_24', 0.04898243954410296, 0.04797542823178382),
 ('RUL_Data_3_25', 0.053994533422334734, 0.05138635686025385),
 ('RUL_Data_3_26', 0.0503310150297394, 0.04787964524450023),
 ('RUL_Data_3_27', 0.0484378804707161, 0.048650409860745404),
 ('RUL_Data_3_28', 0.048531274495484056, 0.04885125432107955),
 ('RUL_Data_3_29', 0.049203263028918635, 0.04879966393357797),
 ('RUL_Data_3_30', 0.048288650068130524, 0.048730819633792256),
 ('RUL_Data_3_31', 0.04734798362990062, 0.04975404481773159),
 ('RUL_Data_3_32', 0.05554109208309486, 0.052354851636400024),
 ('RUL_Data_3_33', 0.04802824106224687, 0.04841498625765149),
 ('RUL_Data_3_34', 0.048353792141010196, 0.04926795423947861),
 ('RUL_Data_3_35', 0.04735417780602482, 0.04832002046840663),
 ('RUL_Data_3_36', 0.048962158577993596, 0.04827298718997514),
 ('RUL_Data_3_37', 0.051799094052907474, 0.050366700954709645),
 ('RUL_Data_3_38', 0.05194964572709141, 0.046769840229645976),
 ('RUL_Data_3_39', 0.04844316147091726, 0.04930087995783165)